In [ ]:
import tensorflow as tf
import DataGenerator
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Input, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from keras.applications.vgg16 import VGG16

num_classes = 96

input_shape = (448, 448, 3)

base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.35)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.35)(x)

bbox_output = Dense(4, activation='linear', name='bboxes_output')(x)

labels_output = Dense(num_classes, activation='softmax', name='labels_output')(x)

model = Model(inputs=base_model.input, outputs=[bbox_output, labels_output])

model.compile(optimizer='rmsprop', loss={'bboxes_output': 'mean_squared_error', 'labels_output': 'categorical_crossentropy'}, metrics={'bboxes_output': 'mse', 'labels_output': 'accuracy'})

print(model.summary())

image_dir = 'output'
annotation_dir = 'output'

batch_size = 8
epochs = 100

train_generator = DataGenerator(image_dir, annotation_dir, batch_size=batch_size, image_size=448)
validation_generator = DataGenerator(image_dir, annotation_dir, batch_size=batch_size, image_size=448)

history = model.fit(train_generator, validation_data=validation_generator, epochs=epochs, steps_per_epoch=len(train_generator), validation_steps=len(validation_generator))

model.save_weights('./results/HCR-Net-Detection.h5')